In [2]:
import pandas as pd
import torch

df = pd.read_csv('train.csv')

df = df.dropna()
df = df.reset_index(drop=True)

In [3]:
import nltk
import unicodedata
import re

def normalize_unicode(s):
    return unicodedata.normalize('NFD', s)

def preprocess_text(text):
    text = normalize_unicode(text)
    text = re.sub(r"(.)(\1{2,})", r"\1", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", " ", text)
    text = text.strip().lower()
    return text

nltk.download('punkt')

df['Description'] = df['Description'].apply(preprocess_text)
df['Description'] = df['Description'].apply(nltk.word_tokenize)
df['Description'] = df['Description'].apply(lambda x: ['<sos>'] + x + ['<eos>'])

df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     /home2/swethavipparla/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,Class Index,Description
0,3,"[<sos>, reuters, short, sellers, ,, wall, stre..."
1,3,"[<sos>, reuters, private, investment, firm, ca..."
2,3,"[<sos>, reuters, soaring, crude, prices, plus,..."
3,3,"[<sos>, reuters, authorities, have, halted, oi..."
4,3,"[<sos>, afp, tearaway, world, oil, prices, ,, ..."


In [4]:
train_set = df['Description']
pre_train_set = list(df['Description'][:8000])
pre_val_set = list(df['Description'][8000:10000])

In [5]:
from torch.utils.data import Dataset

class ELMO_Dataset(Dataset):
    def __init__(self, data, word_to_ix):
        self.data = data
        self.word_to_ix = word_to_ix
        self.indexed_data = [self.index_sentence(sentence) for sentence in data]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.indexed_data[idx])

    def index_sentence(self, sentence):
        indexed_sentence = [self.word_to_ix.get(word, self.word_to_ix['<unk>']) for word in sentence]
        return indexed_sentence

    @staticmethod
    def create_vocab(data):
        vocab = set()
        for sentence in data:
            for word in sentence:
                vocab.add(word)

        vocab.add('<pad>')
        vocab.add('<unk>')
        vocab.add('<sos>')
        vocab.add('<eos>')
        return vocab

In [6]:
word_to_ix = {word: idx for idx, word in enumerate(ELMO_Dataset.create_vocab(train_set))}
print(len(word_to_ix))

59352


In [7]:
from torch.nn.utils.rnn import pad_sequence
def collate_fn(batch):
    batch = sorted(batch, key=lambda x: x.shape[0], reverse=True)
    padded_batch = pad_sequence(batch, batch_first=True, padding_value=word_to_ix['<pad>'])
    lengths = torch.LongTensor([len(x) for x in batch])

    input_tensor = padded_batch[:, :-1]
    target_truth = padded_batch[:, 1:]

    return input_tensor, target_truth, lengths - 1

In [8]:
pre_train_dataset = ELMO_Dataset(pre_train_set, word_to_ix)

pre_val_dataset = ELMO_Dataset(pre_val_set, word_to_ix)

batch_size = 32

from torch.utils.data import DataLoader

pre_train_loader = DataLoader(pre_train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=4)
pre_val_loader = DataLoader(pre_val_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=4)

In [12]:
import gensim
import gensim.downloader

glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


In [13]:
import numpy as np

vocab_size = len(word_to_ix)
embedding_dim = glove_vectors.vector_size
embedding_matrix = torch.zeros(vocab_size, embedding_dim)

special_tokens = ['<pad>', '<unk>', '<sos>', '<eos>']

average_vector = np.mean(glove_vectors.vectors, axis=0)

for word, i in word_to_ix.items():
    if word not in special_tokens:
        try:
            embedding_matrix[i] = torch.tensor(glove_vectors[word])
        except KeyError:
            embedding_matrix[i] = torch.tensor(average_vector)

    elif word == '<sos>' or word == '<eos>':
        embedding_matrix[i] = torch.randn(embedding_dim)
    elif word == '<unk>':
        embedding_matrix[i] = torch.tensor(average_vector)
    else:
        embedding_matrix[i] = torch.zeros(embedding_dim)

print(embedding_matrix.shape)

torch.Size([59352, 50])


In [14]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class ELMO(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, num_layers, dropout):
        super(ELMO, self).__init__()

        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.lstm = nn.LSTM(embedding_matrix.shape[1], hidden_dim, num_layers, dropout=dropout, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(hidden_dim * 2, embedding_matrix.shape[0])

    def forward(self, input_tensor, lengths):
        embedded = self.embedding(input_tensor)
        packed_input = pack_padded_sequence(embedded, lengths, batch_first=True, enforce_sorted=True)
        packed_output, _ = self.lstm(packed_input, None)

        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        output = self.linear(output)

        return output

In [15]:
model = ELMO(embedding_matrix, 300, 2, 0.5)

In [16]:
from tqdm import tqdm

In [17]:
def run_epoch(model, data_loader, loss_fn, epoch, optimizer=None):
    if optimizer:
        model.train()
    else:
        model.eval()

    total_loss = 0

    p_bar = tqdm(data_loader)
    for (input_tensor, target_truth, lengths) in p_bar:

        input_tensor = input_tensor.cuda()
        target_truth = target_truth.cuda()

        output = model(input_tensor, lengths)
        output = output.reshape(-1, output.shape[2])

        loss = loss_fn(output, target_truth.reshape(-1))
        total_loss += loss.item()

        if optimizer:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        mean_loss = total_loss / len(data_loader)

        p_bar.set_description(f'{"T" if optimizer else "V"} Loss: {mean_loss:.4f}, count: {epoch}')


    return mean_loss

In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 50

model.cuda()

best_val_loss = float('inf')

all_val_loss = []
all_train_loss = []


for epoch in range(num_epochs):
    train_loss = run_epoch(model, pre_train_loader, loss_fn, epoch+1, optimizer)
    all_train_loss.append(train_loss)
    with torch.no_grad():
        val_loss = run_epoch(model, pre_val_loader, loss_fn, epoch+1)
        all_val_loss.append(val_loss)

    print('Epoch: {}, Train Loss: {:.4f}, Val Loss: {:.4f}'.format(epoch+1, train_loss, val_loss))
    if val_loss < best_val_loss:
        counter = 0
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_lstm_model.pth')
    else:
        counter += 1
        if counter == 3:
            break

V Loss: 8.5080, count: 1: 100%|██████████| 63/63 [00:02<00:00, 26.35it/s]


Epoch: 1, Train Loss: 8.9388, Val Loss: 8.5080


V Loss: 7.2585, count: 2: 100%|██████████| 63/63 [00:02<00:00, 26.05it/s]


Epoch: 2, Train Loss: 7.8507, Val Loss: 7.2585


V Loss: 6.4858, count: 3: 100%|██████████| 63/63 [00:02<00:00, 26.02it/s]


...

Epoch: 48, Train Loss: 0.0163, Val Loss: 0.6530


V Loss: 0.6441, count: 49: 100%|██████████| 63/63 [00:02<00:00, 25.06it/s]


Epoch: 49, Train Loss: 0.0126, Val Loss: 0.6441


V Loss: 0.6475, count: 50: 100%|██████████| 63/63 [00:02<00:00, 25.46it/s]

Epoch: 50, Train Loss: 0.0108, Val Loss: 0.6475
